In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

from helper_functions import elementlines, nearestlines, theoretical_energy

# %matplotlib inline
# %matplotlib qt

In [ ]:
hs.eds.xray_range('Cu_Ka', 30., hs.material.elements.C.
                  Physical_properties.density_gcm3)

In [ ]:
# files_GaAsCu = ['GaAs_Cu_30kV_50pA_withPulsePileUpCorrection', 'GaAs_Cu_30kV_50pA_noPulsePileUpCorrection']
files_GaAs = ['GaAs_05kV_25pA', 'GaAs_10kV_25pA', 'GaAs_15kV_25pA', 'GaAs_30kV_25pA', 'GaAs_30kV_50pA']
# files_GaSb = ['GaSb_05kV_50pA', 'GaSb_10kV_50pA', 'GaSb_15kV_50pA', 'GaSb_30kV_50pA', 'GaSb_30kV_50pA_noPPUC', 'GaSb_30kV_50pA_processTime1', 
        # 'GaSb_30kV_50pA_processTime2', 'GaSb_30kV_50pA_processTime4', 'GaSb_30kV_400pA_processTime1', 'GaSb_15kV_200pA', 'GaSb_15kV_400pA']
files_GaSb = ['GaSb_05kV_50pA', 'GaSb_10kV_50pA', 'GaSb_15kV_50pA', 'GaSb_30kV_50pA',                          'GaSb_30kV_50pA_processTime1', 
        'GaSb_30kV_50pA_processTime2', 'GaSb_30kV_50pA_processTime4', 'GaSb_30kV_400pA_processTime1', 'GaSb_15kV_200pA', 'GaSb_15kV_400pA']


In [ ]:
# name = 'GaSb'
# files = files_GaSb
# name = 'GaAs'
# files = files_GaAs
# name = 'GaAsCu'
# files = files_GaAsCu

files_all = files_GaSb + files_GaAs
name = 'GaSb+GaAs'
files = files_all

In [ ]:
# # read the lines_info file
lines = [pd.read_csv(f'results/lines_info/{file}_lines_info.csv', index_col=0) for file in files]
lines[1].head(2)

In [ ]:
new_column_names = ['E_0', 'I_b', 'PT', 'ICR', 'DT', 'Live time', 'Duane-Hunt', 'Scale', 'Offset', 'E-res']
old_column_names = ['Nominal beam energy [kV]', 'Beam current [pA]', 'Process time', 'ICR', 'Dead time [%]', 'Live time [s]', 'Duane-Hunt limit [kV]', 'Scale [keV]', 'Offset [keV]', 'Energy resolution [eV]']

In [ ]:
calibrations = [pd.read_csv(f'results/{file}_output.csv', index_col=0) for file in files]
calibs = pd.concat(calibrations, axis=1)
calibs.columns = calibs.iloc[0]
calibs = calibs.drop(calibs.index[0])
calibs = calibs.astype(float)
# limit df precision to 2 decimals
# calibs = calibs.round(2)
calibs.columns = [name.replace('processTime', 'PT') for name in calibs.columns]

df = calibs.T
df = df.rename(columns=dict(zip(old_column_names, new_column_names)))

df.head(2)

In [ ]:
# Duane-Hunt limit
print('Mean Duane-Hunt deviation: ', (df[df['Duane-Hunt']>4]['Duane-Hunt']-df[df['Duane-Hunt']>4]['E_0']).abs().mean())
print('STD Duane-Hunt deviation: ', (df[df['Duane-Hunt']>4]['Duane-Hunt']-df[df['Duane-Hunt']>4]['E_0']).abs().std())
# df[df['Duane-Hunt']>4][['E_0', 'I_b', 'PT', 'Duane-Hunt']]

In [ ]:
# Fiori numbers
fiori_keys = df.keys()[df.keys().str.contains('Fiori')].to_list()

df[['E_0', 'I_b', 'PT'] + fiori_keys]

In [ ]:
# adding column "Percentage that is bg"
df['Bg %'] = df['Background counts'] / df['Total counts']

In [ ]:
line_info_to_add = ['Ga_La', 'As_La', 'Sb_La', 'Ga_Ka', 'As_Ka']

for line in line_info_to_add:
    temp_fwhm = []
    temp_deviation = []
    for l in lines:
        # l is the df of one file
        # now line is e.g. 'Ga_La'
        try:
            temp_fwhm.append(l.loc[line]['FWHM [eV]'])
            temp_deviation.append(l.loc[line]['Delta E [eV]'])
        except KeyError:
            temp_fwhm.append(np.nan)
            temp_deviation.append(np.nan)
    df['FWHM(' + line + ')'] = temp_fwhm
    df['dE(' + line + ')'] = temp_deviation

df

In [ ]:

fwhm_Ga_La = [l.loc['Ga_La']['FWHM [eV]'] for l in lines]
deltaE_Ga_La = [l.loc['Ga_La']['Delta E [eV]'] for l in lines]
df['FWHM [eV] (Ga_La)'] = fwhm_Ga_La
df['Delta E [eV] (Ga_La)'] = deltaE_Ga_La

fwhm_Ga_Ka = []
deltaE_Ga_Ka = []
fwhm_As_La = []
deltaE_As_La = []
fwhm_As_Ka = []
deltaE_As_Ka = []
fwhm_Sb_La = []
deltaE_Sb_La = []


for l in lines:
    try:
        fwhm_Ga_Ka.append(l.loc['Ga_Ka']['FWHM [eV]'])
        deltaE_Ga_Ka.append(l.loc['Ga_Ka']['Delta E [eV]'])
    except KeyError:
        fwhm_Ga_Ka.append(np.nan)
        deltaE_Ga_Ka.append(np.nan)
df['FWHM [eV] (Ga_Ka)'] = fwhm_Ga_Ka
df['Delta E [eV] (Ga_Ka)'] = deltaE_Ga_Ka

# if name == 'GaAs':
#     fwhm_As_La = [l.loc['As_La']['FWHM [eV]'] for l in lines]
#     deltaE_As_La = [l.loc['As_La']['Delta E [eV]'] for l in lines]
#     df['FWHM [eV] (As_La)'] = fwhm_As_La
#     df['Delta E [eV] (As_La)'] = deltaE_As_La

# if name == 'GaSb':
#     fwhm_Sb_La = [l.loc['Sb_La']['FWHM [eV]'] for l in lines]
#     deltaE_Sb_La = [l.loc['Sb_La']['Delta E [eV]'] for l in lines]
#     df['FWHM [eV] (Sb_La)'] = fwhm_Sb_La
#     df['Delta E [eV] (Sb_La)'] = deltaE_Sb_La

In [ ]:
df.to_excel(f'results/healthcheck_statistics_{name}.xlsx')

In [ ]:
# Boxplot of the peak ratios
ratio_keys = ['Ratio (Sb_La/Sb_Lb1)','Ratio (Ga_La/Sb_La)','Ratio (Ga_Ka/Ga_La)','Ratio (Ga_La/As_La)']
# ratio_keys = []
# for key in df.keys():
#     if 'Ratio' in key:
#         ratio_keys.append(key)
    

# make plotly boxplot
fig = go.Figure()
for key in ratio_keys:
    fig.add_trace(go.Box(y=df[key], name=key[5:], boxpoints='all'))
fig.update_layout(title=f'Peak ratios for {name}', yaxis_title='Peak ratio', xaxis_title='Peak')

# plot again, but only if E_0 is 30
for key in ratio_keys:
    fig.add_trace(go.Box(y=df[df['E_0'] == 30][key], name=key[5:] + ' 30kV'))

# y-axis range
fig.update_yaxes(range=[-.2, 3.2])
fig.show()

In [ ]:
# make it as a scatter plot
fig = go.Figure()
for key in ratio_keys:
    fig.add_trace(go.Scatter(x=df['E_0'], y=df[key], mode='markers', name=key[5:]))
fig.update_layout(title=f'Peak ratios for {name}', yaxis_title='Peak ratio', xaxis_title='E_0 [kV]')
fig.show()

In [ ]:
## Looking at the calculated FWHMs from the 30 kV 50 pA files
file_name = 'GaAs_30kV_50pA'
df = pd.read_csv(f'results/lines_info/{file_name}_lines_info.csv', index_col=0)
lines_of_interest = ['Ga_La', 'Ga_Ka', 'As_Ka', 'As_La', 'Ga_Lb1', 'Ga_Kb', 'As_Kb', 'As_Lb1']
df = df.T[lines_of_interest].T
df

In [ ]:
## Looking at the calculated FWHMs from the 30 kV 50 pA files
file_name = 'GaSb_30kV_50pA'
df = pd.read_csv(f'results/lines_info/{file_name}_lines_info.csv', index_col=0)
lines_of_interest = ['Sb_La', 'Ga_Ka', 'Sb_Lb1', 'Ga_La', 'Sb_Lb2', 'Ga_Kb']
df = df.T[lines_of_interest].T
df

In [ ]:
# plot Fiori P/B vs. kV
def plot_fioris(li, vacc):
    for l in li:
        y = []
        for i in range(len(vacc)):
            try:
                y.append(lines[i].T[l]['Fiori P/B'])
            except:
                y.append(np.nan)
        plt.plot(vacc, y, label=l, marker='D')
    plt.legend()
    plt.xlabel('Nominal beam energy [kV]')
    plt.ylabel('Fiori P/B')

In [ ]:
# li=['Ga_La', 'Ga_Ka', 'As_La', 'As_Ka']
# plot_fioris(li, vacc)

In [ ]:
# # plot all fioris
# all_lines = lines[3].T.columns
# plot_fioris(all_lines, vacc)
# plt.show()

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2, sharex=False, sharey=False)
# colors = plt.cm.Blues(np.linspace(0.3, 1, len(vacc)))
# fig.set_size_inches(20, 5)

# for i in range(len(vacc)):
#     ax1.scatter(lines[i]['Calibrated E [keV]'], lines[i]['Fiori P/B'], label=f'{vacc[i]} kV', color=colors[i])
#     ax1.legend()
# for i in range(len(vacc)):
#     ax2.scatter(lines[i]['Calibrated E [keV]'], lines[i]['Fiori P/B'], label=f'{vacc[i]} kV', color=colors[i])
# ax2.set_xlim([0.25, 2])
# ax2.set_ylim([-5, 100])

# ax1.plot([0.25, 2], [100,100], color='gray', linestyle='--', alpha=0.7)
# ax1.plot([0.25, 2], [-5,-5], color='gray', linestyle='--', alpha=0.7)
# ax1.plot([0.25, 0.25], [-5,100], color='gray', linestyle='--', alpha=0.7)
# ax1.plot([2, 2], [-5,100], color='gray', linestyle='--', alpha=0.7)

# fig.supxlabel('keV')
# ax1.set_ylabel('Fiori P/B')
# plt.suptitle('Fiori P/B vs. X-ray energy')